In [1]:
import sys
# sys.path.append('/tf/adv')
from data_reader import DataReader
from rnn_learner import RNNAgent
from util.helper import fix_seeds
from util.logger import LogFile, DLogger
import numpy as np
import pandas as pd

In [2]:
n_folds = 10
confs = []
cells = [5, 8, 10, 15]

for f in range(n_folds):
    for c in range(len(cells)):
        confs.append({'cells': cells[c], 'fold': f})

if __name__ == '__main__':

    n_cells = 5
    
    fold = confs[36]['fold']
    ## Path For GPT
    output_path = 'trained_model/RNN_learner/gpt/cells_' + str(n_cells) + '/fold_' + str(fold) + '/lr_' + str(0.005) + '/'

    ## Path for Gemini 
    # output_path = 'trained_model/RNN_learner/gemini/cells_' + str(n_cells) + '/fold_' + str(fold) + '/lr_' + str(0.005) + '
    
    np.random.seed(1010)
    with LogFile(output_path, 'run.log'):

        data = DataReader.read_synth_nc(data_path = "./img_data/img_gpt_4_vision.csv")
        # data = DataReader.read_synth_nc(data_path = "sim_data/gpt/gpt-4-turbo.csv")

        ## Path for gemini
        # data = DataReader.read_synth_nc(data_path = "./img_data/img_gemmini.csv")

        sh = data['action'].shape

        DLogger.logger().debug('Data dims: ' + str(sh))

        ## For ChatGpt
        # model_path = 'trained_model/inits/gemini/learner_nc_cells_' + str(n_cells) + '/model-init.h5'

        ## for gemini
        model_path = 'trained_model/inits/gemini/learner_nc_cells_' + str(n_cells) + '/model-init.h5'
        agent = RNNAgent(2, 0, n_cells, model_path=model_path)

        indices = np.random.permutation(sh[0])
        folds = np.array_split(indices, n_folds, axis=0)
        training_idx = np.concatenate(folds[(fold+1):] + folds[:fold])
        test_idx = folds[fold]
        ids =   pd.concat([pd.DataFrame({'type': 'train', 'index': training_idx}),
                pd.DataFrame({'type': 'test', 'index': test_idx})])
        ids.to_csv(output_path + 'indices.csv')
        agent.train(
            data['reward'][training_idx],
            data['action'][training_idx],
            data['state'][training_idx] if 'state' in data else None,
            data['reward'][test_idx],
            data['action'][test_idx],
            
            data['state'][test_idx] if 'state' in data else None,
            output_path,
            lr=0.005
        )


2025-06-30 13:57:51,828 - ADV - DEBUG - version control: (None, None)
2025-06-30 13:57:51,849 - ADV - DEBUG - read data from img_gpt_4_vision.csv
2025-06-30 13:57:52,089 - ADV - DEBUG - Data dims: (200, 100)
2025-06-30 13:57:52,451 - ADV - DEBUG - Weights loaded from trained_model/inits/learner_nc_cells_5/model-init.h5
2025-06-30 13:57:52,452 - ADV - DEBUG - Model created with 2 actions and 5 cells
2025-06-30 13:57:52,493 - ADV - DEBUG - Training data dims: (180, 100, 2)
2025-06-30 13:57:52,494 - ADV - DEBUG - Test data dims: (20, 100)
2025-06-30 13:57:52,506 - ADV - DEBUG - Init model weights saved to: trained_model/RNN_learner/cells_5/fold_9/lr_0.005/model-init.weights.h5
2025-06-30 13:57:52,959 - ADV - DEBUG - step 0: mean test loss = 0.6921477317810059, %correct = 0.5285000205039978
2025-06-30 13:57:52,964 - ADV - DEBUG - Model weights saved to: trained_model/RNN_learner/cells_5/fold_9/lr_0.005/model-0.weights.h5
2025-06-30 13:57:53,700 - ADV - DEBUG - step 0: mean loss = 12472.370

##### 12:23 pm